# Sweeps
familiarizing with wandb sweeps for automatic hyperparameter tuning

In [1]:
%cd /home/ltchen/gnnpp
import sys
import os
import pytorch_lightning as L
import torch
import torch_geometric
import json
import wandb

# from typing import Tuple
# from torch_geometric.nn import GATv2Conv
# from torch_geometric.utils import scatter
# from torch.nn import Linear, ModuleList, ReLU
from torch_geometric.loader import DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, TQDMProgressBar
from torch.optim import AdamW
from pytorch_lightning.loggers import WandbLogger

# from models.loss import NormalCRPS
# from models.model_utils import MakePositive, EmbedStations
from utils.data import (
    load_dataframes,
    summary_statistics,
)
from exploration.graph_creation import *
from models.graphensemble.multigraph import *

/home/ltchen/gnnpp


In [15]:

sweep_config = {
    'method': 'random'
    }

In [5]:
metric = {
    'name': 'val_loss', # train_loss?
    'goal': 'minimize'
    }

sweep_config['metric'] = metric

In [31]:
parameters_dict = {
    'optimizer': {
        'value': 'adam'
        },
    # 'batch_size': {
    #     'value': 8
    #     },
    'gnn_hidden': {
          'values': [64, 128, 265]
        },
    'gnn_layers': {
          'values': [1, 2]
        },
    'heads': {
          'value': 8
        },
    'lr': {
        'distribution': 'uniform',
        'min': 0,
        'max': 0.001
      },
    # 'max_dist': { # might not even need it?
    #       'value': 50
    #     },
    # 'max_epochs': { # early stopping?
    #       'value': 100
    #     },
    }

sweep_config['parameters'] = parameters_dict


In [7]:
import pprint
pprint.pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'batch_size': {'values': 8},
                'gnn_hidden': {'values': [64, 128, 265]},
                'gnn_layers': {'values': [1, 2]},
                'heads': {'values': 8},
                'lr': {'distribution': 'uniform', 'max': 0.1, 'min': 0},
                'max_dist': {'values': 50},
                'max_epochs': {'values': 100},
                'optimizer': {'values': ['adam']}}}


In [ ]:
'''{"batch_size":8,
"gnn_hidden":256,
"gnn_layers":1,
"heads":8,
"lr":0.0001,
"max_dist":50,
"max_epochs": 23,
"remove_edges": "False",
"only_summary": "True"}'''

In [17]:
sweep_id = wandb.sweep(sweep_config, project="sweep_test")

Create sweep with ID: jz97fbz2
Sweep URL: https://wandb.ai/leachen01-karlsruhe-institute-of-technology/sweep_test/sweeps/jz97fbz2


In [19]:
leadtime = "24h"

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
DIRECTORY = os.getcwd()
JSONPATH = os.path.join(DIRECTORY, f"trained_models/no_ensemble_{leadtime}/params.json")
with open(JSONPATH, "r") as f:
    print(f"[INFO] Loading {JSONPATH}")
    args_dict = json.load(f)
# config = args_dict

[INFO] Loading /home/ltchen/gnnpp/trained_models/no_ensemble_24h/params.json


In [20]:
dataframes = load_dataframes(leadtime=leadtime)
dataframes = summary_statistics(dataframes)

[INFO] Dataframes exist. Will load pandas dataframes.
[INFO] Calculating summary statistics for train
[INFO] Calculating summary statistics for valid
[INFO] Calculating summary statistics for test_rf
[INFO] Calculating summary statistics for test_f


In [29]:
graph_name = "g3"
SAVEPATH = os.path.join(DIRECTORY, f"leas_trained_models/sum_stats_{leadtime}/{graph_name}_{leadtime}/models")
RESULTPATH = os.path.join(DIRECTORY, f"leas_trained_models/sum_stats_{leadtime}/{graph_name}_{leadtime}")

graphs3_train_rf, tests3 = normalize_features_and_create_graphs1(df_train=dataframes['train'], df_valid_test=[dataframes['valid'], dataframes['test_rf'], dataframes['test_f']], station_df=dataframes['stations'], attributes=["geo", "alt", "lon", "lat", "alt-orog"], edges=[("geo", 100), ("alt", 10), ("alt-orog", 4)], sum_stats = True)
graphs3_valid_rf, graphs3_test_rf, graphs3_test_f = tests3

g3_train_loader = DataLoader(graphs3_train_rf, batch_size=8, shuffle=True)
g3_valid_loader = DataLoader(graphs3_valid_rf, batch_size=8, shuffle=False)
# g3_test_f_loader = DataLoader(graphs3_test_f, batch_size=config['batch_size'], shuffle=False)
# g3_test_rf_loader = DataLoader(graphs3_test_rf, batch_size=config['batch_size'], shuffle=False)

train_loader = g3_train_loader
valid_loader = g3_valid_loader
# test_f_loader = g3_test_f_loader
# test_rf_loader = g3_test_rf_loader
# test_loader = [test_f_loader, test_rf_loader]

emb_dim = 20
in_channels = graphs3_train_rf[0].x.shape[1] + emb_dim - 1
edge_dim = graphs3_train_rf[0].num_edge_features
num_nodes = graphs3_train_rf[0].num_nodes
# max_epochs = max_epoch_list[graph_name]
max_epochs = 100

facts_about(graphs3_train_rf[0])

# embedding_dim = emb_dim
# in_channels = in_channels
# hidden_channels_gnn = config['gnn_hidden']
# out_channels_gnn = config['gnn_hidden']
# num_layers_gnn = config['gnn_hidden']
# heads = config['heads']
# hidden_channels_deepset = config['gnn_hidden']
# optimizer_class = AdamW
# optimizer_params = dict(lr=config['lr'])

[INFO] Normalizing features...
fit_transform
transform 1
transform 2
transform 3


100%|██████████| 730/730 [00:02<00:00, 264.68it/s]

Number of nodes: 120 with feature dimension of x: 65
Number of isolated nodes: 1
Number of edges: 2626 with edge dimension: 5
Average node degree: 21.883333206176758


In [36]:
PROJECTNAME = "sweep_test"

TRAINNAME = f"{graph_name}_{leadtime}_train_run1"

with wandb.init():
    config = wandb.config

    multigraph = Multigraph(
        num_nodes=num_nodes,  #
        embedding_dim=emb_dim,
        edge_dim=edge_dim,
        in_channels=in_channels,
        hidden_channels_gnn=config['gnn_hidden'],
        out_channels_gnn=config['gnn_hidden'],
        num_layers_gnn=config['gnn_layers'],
        heads=config['heads'],
        hidden_channels_deepset=config['gnn_hidden'],
        optimizer_class=AdamW,
        optimizer_params=dict(lr=config['lr']),
    )
    # torch.compile(multigraph)
    # batch = next(iter(train_loader))
    # multigraph.forward(batch)
    multigraph.initialize(train_loader)

    wandb_logger = WandbLogger(project=PROJECTNAME)
    early_stop = EarlyStopping(monitor="val_loss", patience=10)
    # progress_bar = TQDMProgressBar(refresh_rate=0)

    # checkpoint_callback = ModelCheckpoint(
    #     dirpath=SAVEPATH, filename=TRAINNAME, monitor="val_loss", mode="min", save_top_k=1
    # )

    trainer = L.Trainer(
        max_epochs=1000,
        log_every_n_steps=1,
        accelerator="gpu",
        devices=1,
        enable_progress_bar=True,
        logger=wandb_logger,
        # callbacks=[early_stop, progress_bar, checkpoint_callback],
        callbacks=early_stop,
    )

    trainer.fit(model=multigraph, train_dataloaders=train_loader, val_dataloaders=valid_loader)

Traceback (most recent call last):
  File "/tmp/ipykernel_539441/2930110546.py", line 10, in <module>
    multigraph = Multigraph(
  File "/tmp/pycharm_project_408/models/graphensemble/multigraph.py", line 128, in __init__
    self.conv = ResGnn(
  File "/tmp/pycharm_project_408/models/graphensemble/multigraph.py", line 36, in __init__
    assert num_layers > 0, "num_layers must be > 0."
TypeError: '>' not supported between instances of 'dict' and 'int'


TypeError: '>' not supported between instances of 'dict' and 'int'

In [ ]:
wandb.agent(sweep_id, train, count=5)

In [ ]:
'''with wandb.init():
    config = wandb.config
    print("[INFO] Starting sweep with config: ", config)

    multigraph = Multigraph(
        embedding_dim=emb_dim,
        in_channels=in_channels,
        hidden_channels_gnn=config.gnn_hidden,
        out_channels_gnn=config.gnn_hidden,
        num_layers_gnn=config.gnn_layers,
        heads=config.heads,
        hidden_channels_deepset=config.gnn_hidden,
        optimizer_class=AdamW,
        optimizer_params=dict(lr=config.lr),
    )
    torch_geometric.compile(multigraph)

    # run a dummy forward pass to initialize the model
    multigraph.initialize(train_loader)

    wandb_logger = WandbLogger(project="multigraph")
    early_stop = EarlyStopping(monitor="val_loss", patience=10)

    # Train Model ###################################################################
    print("[INFO] Training model...")
    trainer = L.Trainer(
        max_epochs=1000,
        log_every_n_steps=1,
        accelerator="gpu",
        enable_progress_bar=True,
        logger=wandb_logger,
        callbacks=early_stop,
    )

    trainer.fit(model=multigraph, train_dataloaders=train_loader, val_dataloaders=valid_loader)'''